In [2]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1260
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [12]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate() + LoggerMessageGate())
    
    x_prev_add = add!(model, AdditionNode())
    
    x_prior = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1   = add!(model, constvar(:add_1, 1.0))

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    
    add_x_and_noise = add!(model, AdditionNode())
    
    x = add!(model, simplerandomvar(:x))
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = add!(model, datavar(:y, Float64))
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(model, x_prev_add)
    activate!(model, add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()

    link(x_prior, getmarginal(x))
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [13]:
N = 10
data = collect(1:N) + sqrt(200.0) * randn(N);

In [14]:
kalman(data)

From variable out of node typeof(+) => Normal{Float64}(μ=1.0, σ=100.0)
From variable in1 of node typeof(+) => Normal{Float64}(μ=10.610286371278645, σ=14.142135623730951)
From variable out of node typeof(+) => Normal{Float64}(μ=11.421849383606515, σ=14.002800840280099)
From variable in1 of node typeof(+) => Normal{Float64}(μ=5.610347863463353, σ=14.142135623730951)
From variable out of node typeof(+) => Normal{Float64}(μ=9.544868433040595, σ=9.950371902099892)
From variable in1 of node typeof(+) => Normal{Float64}(μ=11.058334949499901, σ=14.142135623730951)
From variable out of node typeof(+) => Normal{Float64}(μ=11.046016286172817, σ=8.137884587711596)
From variable in1 of node typeof(+) => Normal{Float64}(μ=9.70243491851462, σ=14.142135623730951)
From variable out of node typeof(+) => Normal{Float64}(μ=11.71179206536232, σ=7.053456158585984)
From variable in1 of node typeof(+) => Normal{Float64}(μ=-13.458088318224597, σ=14.142135623730951)
From variable out of node typeof(+) => Normal

10-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=10.421849383606515, σ=14.002800840280099)
 Normal{Float64}(μ=8.544868433040595, σ=9.950371902099892)
 Normal{Float64}(μ=10.046016286172817, σ=8.137884587711596)
 Normal{Float64}(μ=10.71179206536232, σ=7.053456158585984)
 Normal{Float64}(μ=6.697871670225484, σ=6.311944030978033)
 Normal{Float64}(μ=3.654648648449137, σ=5.763904177042351)
 Normal{Float64}(μ=6.03761928964734, σ=5.337605126836238)
 Normal{Float64}(μ=8.213216799770384, σ=4.993761694389224)
 Normal{Float64}(μ=8.595409453788013, σ=4.708816093480111)
 Normal{Float64}(μ=9.709906628415208, σ=4.467670516087703)